# Création de prompts pour le nommage et la description des clusters

In [2]:
Sys.setlocale("LC_ALL", "fr_CA.UTF-8")

# Library
library(dplyr)
library(tidyr)
library(ggplot2)
library(ggtext)
library(tidytext)
library(ellipsellm)

# Load Data and Clustering Results
kmeans_result <- readRDS("../../_PrivateFolder_datagotchi_federal_2025/data/clustering/can2025/03_pilot_kmeans.rds") # nolint
dfUsedForClustering <- readRDS("../../_PrivateFolder_datagotchi_federal_2025/data/clustering/can2025/03_pilot_2025.rds")
df_clusters_composition <- readRDS("../../_PrivateFolder_datagotchi_federal_2025/data/clustering/can2025/03_pilot_clusters_composition.rds")

[1] "fr_CA.UTF-8/fr_CA.UTF-8/fr_CA.UTF-8/C/fr_CA.UTF-8/C"

In [3]:
cluster_names <- unique(dfUsedForClustering$cluster_name)
cluster_names <- cluster_names[order(cluster_names)]

prompts <- list()

for (cluster_value in cluster_names) {
  cluster_data <- dfUsedForClustering %>% filter(cluster_name == cluster_value)
  cluster_vars <- df_clusters_composition %>% filter(cluster_name == cluster_value)

  # Calculer quelques statistiques démographiques pour le cluster
  perc_hommes     <- mean(cluster_data$ses_genderMale, na.rm = TRUE) * 100
  perc_immigrants <- mean(cluster_data$ses_immigrant, na.rm = TRUE) * 100
  perc_langEn     <- mean(cluster_data$ses_languageEnglish, na.rm = TRUE) * 100
  perc_langFr     <- mean(cluster_data$ses_languageFrench, na.rm = TRUE) * 100
  perc_ses_languageOther <- mean(cluster_data$ses_languageOther, na.rm = TRUE) * 100
  perc_ethn_white <- mean(cluster_data$ses_ethnicityWhite, na.rm = TRUE) * 100
  perc_hetero     <- mean(cluster_data$ses_sexOrientationHetero, na.rm = TRUE) * 100
  educ_mean_BHS       <- mean(cluster_data$ses_educBHS, na.rm = TRUE) * 100
  educ_mean_PostHS       <- mean(cluster_data$ses_educPostHS, na.rm = TRUE) * 100
  educ_mean_Univ       <- mean(cluster_data$ses_educUniv, na.rm = TRUE) * 100
  age_mean        <- mean(cluster_data$ses_age, na.rm = TRUE) * 80
  revenu_moyen_Low    <- mean(cluster_data$ses_incomeLow, na.rm = TRUE) * 100
  revenu_moyen_Mid    <- mean(cluster_data$ses_incomeMid, na.rm = TRUE) * 100
  revenu_moyen_High    <- mean(cluster_data$ses_incomeHigh, na.rm = TRUE) * 100
  perc_pecheurs        <- mean(cluster_data$lifestyle_goFishingFreq_bin, na.rm = TRUE) * 100
  perc_chasseurs        <- mean(cluster_data$lifestyle_goHuntingFreq_bin, na.rm = TRUE) * 100
  perc_motorized        <- mean(cluster_data$lifestyle_motorizedActFreq_bin, na.rm = TRUE) * 100
  perc_unmotorized   <- mean(cluster_data$lifestyle_unmotorizedActFreq_bin, na.rm = TRUE) * 100
  perc_tofu   <- mean(cluster_data$lifestyle_fridgeTofuTempeh, na.rm = TRUE) * 100
  perc_tattoos  <- mean(cluster_data$lifestyle_hasTattoos, na.rm = TRUE) * 100

  # Génération du prompt de suggestion de noms
  prompt_description <- (paste0(
  "Propose un prénom pour une personne habitant au Canada et donne une brève description pour le persona ", cluster_value, " :\n\n",
  
  "Variables avec une moyenne significativement plus élevée que la moyenne de la population en général :\n",
  if (any(cluster_vars$color == "green")) {
    paste(cluster_vars[cluster_vars$color == "green", "variable"], collapse = "\n")
  } else {
    "Aucune variable n'a une moyenne significativement plus élevée que la moyenne de la population en général."
  },
  
  "\n\nVariables avec une moyenne significativement plus basse que la moyenne de la population en général :\n",
  if (any(cluster_vars$color == "red")) {
    paste(cluster_vars[cluster_vars$color == "red", "variable"], collapse = "\n")
  } else {
    "Aucune variable avec une moyenne significativement plus basse que la moyenne de la population en général."
  },
  
  "\n\nCe cluster est composé de :\n", 
  "Hommes: ", round(perc_hommes, 1), "%, ",
  "Immigrants: ", round(perc_immigrants, 1), "%\n",
  "Âge: ", round(age_mean, 1), " ans\n",
  "Revenus:\n",
  "  Revenu Bas: ", round(revenu_moyen_Low, 1), " %\n",
  "  Revenu Mid: ", round(revenu_moyen_Mid, 1), " %\n",
  "  Revenu High: ", round(revenu_moyen_High, 1), " %\n",
  "Langues:\n",
  "  En: ", round(perc_langEn, 1), "%, ",
  "Fr: ", round(perc_langFr, 1), "%, ",
  "Autres: ", round(perc_ses_languageOther, 1), "%\n",
  "EducBHS: ", round(educ_mean_BHS, 1), "%\n",
  "EducPostHS: ", round(educ_mean_PostHS, 1), "%\n",
  "EducUniv: ", round(educ_mean_Univ, 1), "%\n",
  "Ethnie (Blanc): ", round(perc_ethn_white, 1), "%\n",
  "Orientation (Hétéro): ", round(perc_hetero, 1), "%\n",
  "Pêcheurs: ", round(perc_pecheurs, 1), "%\n",
  "Chasseurs: ", round(perc_chasseurs, 1), "%\n",
  "Font du sport motorisé: ", round(perc_motorized, 1), "%\n",
  "Font du sport non-motorisé: ", round(perc_unmotorized, 1), "%\n",
  "Ont du tofu dans leur frigo: ", round(perc_tofu, 1), "%\n",
  "Tattoos: ", round(perc_tattoos, 1), "%\n"
))

  prompts[[cluster_value]] <- prompt_description
}


Warning message in mean.default(cluster_data$lifestyle_goFishingFreq_bin, na.rm = TRUE):
“l'argument n'est ni numérique, ni logique : renvoi de NA”
Warning message in mean.default(cluster_data$lifestyle_goHuntingFreq_bin, na.rm = TRUE):
“l'argument n'est ni numérique, ni logique : renvoi de NA”
Warning message in mean.default(cluster_data$lifestyle_motorizedActFreq_bin, na.rm = TRUE):
“l'argument n'est ni numérique, ni logique : renvoi de NA”
Warning message in mean.default(cluster_data$lifestyle_unmotorizedActFreq_bin, na.rm = TRUE):
“l'argument n'est ni numérique, ni logique : renvoi de NA”
Warning message in mean.default(cluster_data$lifestyle_fridgeTofuTempeh, na.rm = TRUE):
“l'argument n'est ni numérique, ni logique : renvoi de NA”
Warning message in mean.default(cluster_data$lifestyle_goFishingFreq_bin, na.rm = TRUE):
“l'argument n'est ni numérique, ni logique : renvoi de NA”
Warning message in mean.default(cluster_data$lifestyle_goHuntingFreq_bin, na.rm = TRUE):
“l'argument n'es

In [4]:
prompts

$`1`
[1] "Propose un prénom pour une personne habitant au Canada et donne une brève description pour le persona 1 :\n\nVariables avec une moyenne significativement plus élevée que la moyenne de la population en général :\nc(\"dv_voteChoiceOther\", \"ses_age\", \"ses_educBHS\", \"ses_incomeLow\", \"ses_languageFrench\", \"ses_regionQuebec\")\n\nVariables avec une moyenne significativement plus basse que la moyenne de la population en général :\nc(\"dv_solidity\", \"dv_turnout\", \"dv_voteChoiceBQ\", \"dv_voteChoiceCPC\", \"dv_voteChoiceLPC\", \"dv_voteChoiceNDP\", \"ses_educUniv\", \"ses_incomeHigh\", \"ses_regionOntario\", \"ses_urban\")\n\nCe cluster est composé de :\nHommes: 44%, Immigrants: 10%\nÂge: 56 ans\nRevenus:\n  Revenu Bas: 100 %\n  Revenu Mid: 0 %\n  Revenu High: 0 %\nLangues:\n  En: 42%, Fr: 54%, Autres: 4%\nEducBHS: 62%\nEducPostHS: 32%\nEducUniv: 6%\nEthnie (Blanc): 84%\nOrientation (Hétéro): 96%\nPêcheurs: NA%\nChasseurs: NA%\nFont du sport motorisé: NA%\nFont du sport non-motorisé: NA%\nOnt du tofu dans leur frigo: NA%\nTattoos: 30%\n"

$`10`
[1] "Propose un prénom pour une personne habitant au Canada et donne une brève description pour le persona 10 :\n\nVariables avec une moyenne significativement plus élevée que la moyenne de la population en général :\nc(\"dv_turnout\", \"dv_voteChoiceNDP\", \"ses_regionWest\")\n\nVariables avec une moyenne significativement plus basse que la moyenne de la population en général :\nc(\"dv_voteChoiceBQ\", \"dv_voteChoiceCPC\", \"dv_voteChoiceLPC\", \"dv_voteChoiceOther\", \"ses_languageFrench\", \"ses_regionOntario\", \"ses_regionQuebec\")\n\nCe cluster est composé de :\nHommes: 41%, Immigrants: 18%\nÂge: 40 ans\nRevenus:\n  Revenu Bas: 14.8 %\n  Revenu Mid: 63.9 %\n  Revenu High: 21.3 %\nLangues:\n  En: 95.1%, Fr: 1.6%, Autres: 3.3%\nEducBHS: 24.6%\nEducPostHS: 34.4%\nEducUniv: 41%\nEthnie (Blanc): 72.1%\nOrientation (Hétéro): 67.2%\nPêcheurs: NA%\nChasseurs: NA%\nFont du sport motorisé: NA%\nFont du sport non-motorisé: NA%\nOnt du tofu dans leur frigo: NA%\nTattoos: 47.5%\n"

$`11`
[1] "Propose un prénom pour une personne habitant au Canada et donne une brève description pour le persona 11 :\n\nVariables avec une moyenne significativement plus élevée que la moyenne de la population en général :\nc(\"dv_voteChoiceLPC\", \"ses_educUniv\", \"ses_regionOntario\", \"ses_urban\")\n\nVariables avec une moyenne significativement plus basse que la moyenne de la population en général :\nc(\"dv_voteChoiceBQ\", \"dv_voteChoiceCPC\", \"dv_voteChoiceNDP\", \"dv_voteChoiceOther\", \"ses_languageFrench\", \"ses_regionQuebec\")\n\nCe cluster est composé de :\nHommes: 48.3%, Immigrants: 19.7%\nÂge: 48.7 ans\nRevenus:\n  Revenu Bas: 11.6 %\n  Revenu Mid: 76.2 %\n  Revenu High: 12.2 %\nLangues:\n  En: 93.9%, Fr: 1.4%, Autres: 4.8%\nEducBHS: 17%\nEducPostHS: 27.2%\nEducUniv: 55.8%\nEthnie (Blanc): 71.4%\nOrientation (Hétéro): 85%\nPêcheurs: NA%\nChasseurs: NA%\nFont du sport motorisé: NA%\nFont du sport non-motorisé: NA%\nOnt du tofu dans leur frigo: NA%\nTattoos: 25.2%\n"

$`12`
[1] "Propose un prénom pour une personne habitant au Canada et donne une brève description pour le persona 12 :\n\nVariables avec une moyenne significativement plus élevée que la moyenne de la population en général :\nc(\"dv_voteChoiceOther\", \"ses_languageFrench\", \"ses_regionQuebec\")\n\nVariables avec une moyenne significativement plus basse que la moyenne de la population en général :\nc(\"dv_solidity\", \"dv_voteChoiceBQ\", \"dv_voteChoiceCPC\", \"dv_voteChoiceLPC\", \"dv_voteChoiceNDP\", \"ses_incomeLow\", \"ses_regionOntario\", \"ses_regionWest\")\n\nCe cluster est composé de :\nHommes: 36.4%, Immigrants: 13.6%\nÂge: 48.5 ans\nRevenus:\n  Revenu Bas: 0 %\n  Revenu Mid: 84.8 %\n  Revenu High: 15.2 %\nLangues:\n  En: 7.6%, Fr: 92.4%, Autres: 0%\nEducBHS: 28.8%\nEducPostHS: 30.3%\nEducUniv: 40.9%\nEthnie (Blanc): 95.5%\nOrientation (Hétéro): 93.9%\nPêcheurs: NA%\nChasseurs: NA%\nFont du sport motorisé: NA%\nFont

In [5]:
reponses_openai <- list()
# Afficher ou traiter tous les prompts
for (cluster in names(prompts)) {
  # Par exemple, envoyer chaque prompt à OpenAI (si la fonction est correctement définie)
  conv <- openai_create_conversation(prompts[[cluster]])
  resp <- openai_chat_completion(conv)
  reponses_openai[[cluster]] <- resp$content
}


In [6]:
reponses_openai

$`1`
[1] "Pour une personne vivant au Canada, spécifiquement au Québec, un prénom approprié pourrait être \"Réjean\". Voici une brève description pour le persona correspondant aux variables fournies :\n\n### Prénom : Réjean\n\n#### Description :\n\nRéjean est un homme francophone de 56 ans vivant dans une région rurale du Québec. Il a un diplôme d'études secondaires et n'a pas poursuivi d'études universitaires. Son revenu est bas, et il vit avec des moyens modestes. Réjean parle principalement le français, avec quelques connaissances en anglais.\n\nIl est peu engagé politiquement, ne s'identifiant pas fortement à un parti politique traditionnel comme le Bloc Québécois, le Parti conservateur, le Parti libéral ou le NPD. Son choix de vote pourrait donc souvent se tourner vers des partis alternatifs ou indépendants.\n\nRéjean a tendance à ne pas voter régulièrement, reflétant peut-être un sentiment de détachement par rapport aux enjeux politiques. Il vit dans une région rurale ou semi-rurale et non pas en milieu urbain. \n\nBien qu'il n'ait pas de revenu élevé, Réjean trouve de la satisfaction dans les petites choses de la vie. Il pourrait être intéressé par des activités qui ne nécessitent pas beaucoup de ressources financières. Un tiers de ses pairs ont des tatouages, ce qui pourrait suggérer une certaine ouverture d'esprit ou un désir d'expression personnelle.\n\nEn somme, Réjean est un homme simple, ancré dans sa communauté locale et francophone, avec une vie modeste mais riche en culture québécoise."

$`10`
[1] "Pour une personne habitant au Canada correspondant au persona 10, je propose le prénom \"Liam\".\n\n**Description du persona :**\n\n**Nom :** Liam\n\n**Âge :** 40 ans\n\n**Genre :** Homme (probabilité de 41%)\n\n**Origine :** Principalement d'origine blanche (72.1%), avec une faible probabilité d'être immigrant (18%).\n\n**Région de résidence :** Ouest du Canada. Liam est plus susceptible de vivre dans une région de l'Ouest canadien, ce qui est cohérent avec son soutien au NPD (Nouveau Parti Démocratique), et moins susceptible de vivre en Ontario ou au Québec.\n\n**Langue :** Liam parle principalement anglais (95.1%), avec très peu de chances de parler français (1.6%).\n\n**Éducation :** Liam a un bon niveau d'éducation, avec 41% ayant un diplôme universitaire. Il est également probable qu'il ait terminé des études postsecondaires (34.4%).\n\n**Revenu :** Liam fait partie de la classe moyenne, avec 63.9% ayant un revenu moyen. Il y a aussi une probabilité modérée qu'il ait un revenu élevé (21.3%).\n\n**Orientation sexuelle :** Principalement hétérosexuel (67.2%).\n\n**Engagement politique :** Liam a une forte probabilité de voter, surtout pour le NPD, et il a une faible probabilité de voter pour le Bloc Québécois, le Parti conservateur, le Parti libéral, ou d'autres partis.\n\n**Caractéristiques supplémentaires :** Liam pourrait avoir des tatouages (47.5%), ce qui peut indiquer une certaine ouverture à l'art corporel ou une expression personnelle.\n\nLiam est probablement une personne engagée dans sa communauté, avec un intérêt pour les questions sociales et progressistes, ce qui peut se refléter dans ses choix politiques et son niveau d'éducation."

$`11`
[1] "Pour une personne vivant au Canada correspondant au persona 11, je propose le prénom \"Ethan\". Ce prénom est courant en Ontario et correspond bien à une personne anglophone.\n\n### Description du Persona 11 : Ethan\n\n- **Prénom:** Ethan\n- **Âge:** 49 ans\n- **Sexe:** Homme (probabilité légèrement plus élevée que femme)\n- **Origine:** Principalement d'origine ethnique blanche\n- **Langue principale:** Anglais\n- **Éducation:** Diplômé universitaire\n- **Région:** Vit en Ontario, probablement dans une zone urbaine\n- **Statut d'immigrant:** 19.7%, donc probable qu'Ethan soit né au Canada, avec une possibilité qu'il soit immigrant\n- **Situation socio-économique:** \n  - Revenu moyen (76.2% dans la catégorie de revenu moyen)\n  - Appartient à la classe moyenn

In [7]:
description_clusters <- data.frame(prompt = unlist(prompts), description = unlist(reponses_openai))

In [8]:
description_clusters

,prompt,description
,<chr>,<chr>
1,"Propose un prénom pour une personne habitant au Canada et donne une brève description pour le persona 1 : Variables avec une moyenne significativement plus élevée que la moyenne de la population en général : c(""dv_voteChoiceOther"", ""ses_age"", ""ses_educBHS"", ""ses_incomeLow"", ""ses_languageFrench"", ""ses_regionQuebec"") Variables avec une moyenne significativement plus basse que la moyenne de la population en général : c(""dv_solidity"", ""dv_turnout"", ""dv_voteChoiceBQ"", ""dv_voteChoiceCPC"", ""dv_voteChoiceLPC"", ""dv_voteChoiceNDP"", ""ses_educUniv"", ""ses_incomeHigh"", ""ses_regionOntario"", ""ses_urban"") Ce cluster est composé de : Hommes: 44%, Immigrants: 10% Âge: 56 ans Revenus: Revenu Bas: 100 % Revenu Mid: 0 % Revenu High: 0 % Langues: En: 42%, Fr: 54%, Autres: 4% EducBHS: 62% EducPostHS: 32% EducUniv: 6% Ethnie (Blanc): 84% Orientation (Hétéro): 96% Pêcheurs: NA% Chasseurs: NA% Font du sport motorisé: NA% Font du sport non-motorisé: NA% Ont du tofu dans leur frigo: NA% Tattoos: 30%","Pour une personne vivant au Canada, spécifiquement au Québec, un prénom approprié pourrait être ""Réjean"". Voici une brève description pour le persona correspondant aux variables fournies : ### Prénom : Réjean #### Description : Réjean est un homme francophone de 56 ans vivant dans une région rurale du Québec. Il a un diplôme d'études secondaires et n'a pas poursuivi d'études universitaires. Son revenu est bas, et il vit avec des moyens modestes. Réjean parle principalement le français, avec quelques connaissances en anglais. Il est peu engagé politiquement, ne s'identifiant pas fortement à un parti politique traditionnel comme le Bloc Québécois, le Parti conservateur, le Parti libéral ou le NPD. Son choix de vote pourrait donc souvent se tourner vers des partis alternatifs ou indépendants. Réjean a tendance à ne pas voter régulièrement, reflétant peut-être un sentiment de détachement par rapport aux enjeux politiques. Il vit dans une région rurale ou semi-rurale et non pas en milieu urbain. Bien qu'il n'ait pas de revenu élevé, Réjean trouve de la satisfaction dans les petites choses de la vie. Il pourrait être intéressé par des activités qui ne nécessitent pas beaucoup de ressources financières. Un tiers de ses pairs ont des tatouages, ce qui pourrait suggérer une certaine ouverture d'esprit ou un désir d'expression personnelle. En somme, Réjean est un homme simple, ancré dans sa communauté locale et francophone, avec une vie modeste mais riche en culture québécoise."
10,"Propose un prénom pour une personne habitant au Canada et donne une brève description pour le persona 10 : Variables avec une moyenne significativement plus élevée que la moyenne de la population en général : c(""dv_turnout"", ""dv_voteChoiceNDP"", ""ses_regionWest"") Variables avec une moyenne significativement plus basse que la moyenne de la population en général : c(""dv_voteChoiceBQ"", ""dv_voteChoiceCPC"", ""dv_voteChoiceLPC"", ""dv_voteChoiceOther"", ""ses_languageFrench"", ""ses_regionOntario"", ""ses_regionQuebec"") Ce cluster est composé de : Hommes: 41%, Immigrants: 18% Âge: 40 ans Revenus: Revenu Bas: 14.8 % Revenu Mid: 63.9 % Revenu High: 21.3 % Langues: En: 95.1%, Fr: 1.6%, Autres: 3.3% EducBHS: 24.6% EducPostHS: 34.4% EducUniv: 41% Ethnie (Blanc): 72.1% Orientation (Hétéro): 67.2% Pêcheurs: NA% Chasseurs: NA% Font du sport motorisé: NA% Font du sport non-motorisé: NA% Ont du tofu dans leur frigo: NA% Tattoos: 47.5%","Pour une personne habitant au Canada correspondant au persona 10, je propose le prénom ""Liam"". **Description du persona :** **Nom :** Liam **Âge :** 40 ans **Genre :** Homme (probabilité de 41%) **Origine :** Principalement d'origine blanche (72.1%), avec une faible probabilité d'être immigrant (18%). **Région de résidence :** Ouest du Canada. Liam est plus susceptible de vivre dans une région de l'Ouest canadien, ce qui est cohérent avec son soutien au NPD (Nouveau Parti Démocratique), et mo

In [9]:
write.csv(description_clusters, "description_clusters.csv", row.names = FALSE)

In [10]:
saveRDS(description_clusters, file = "../../_PrivateFolder_datagotchi_federal_2025/data/clustering/can2025/03_pilot_clusters_description.rds")